## Proceso de Scrapping de Sentimientos


In [7]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import pandas as pd
import datetime



In [8]:
# creo api para hacer las llamadas
api = API()

### Defino funciones

In [9]:
def save_info(dataset,filename):
    df = pd.DataFrame(dataset)
    df.to_csv(filename, index=False)

In [10]:
def read_and_concat(ruta,data):
    df_new = pd.DataFrame(data)
    df = pd.read_csv(ruta)
    df_actualizado = pd.concat([df, df_new], ignore_index=True)
    df_actualizado.to_csv(ruta, index=False)

In [11]:
def try_read_and_concat(ruta,data):
    df_new = pd.DataFrame(data)
    try: 
        df = pd.read_csv(ruta)
        df_actualizado = pd.concat([df, df_new], ignore_index=True)
        df_actualizado.to_csv(ruta, index=False)
    except:
        df = pd.DataFrame(data)
        df.to_csv(ruta, index=False)
   

In [17]:
async def get_top_tweets(keywords,limit=1000):
    tweets = await gather(api.search(keywords, limit, kv={"product": "Top"}))
    return tweets

In [37]:
def parse_sentiment_tweets(tweets,sentimiento="",emocion="",categoria="Tweet"):
    parsed_tweets = []
    parsed_users = []
    existing_ids = []
    for tweet in tweets:
            data_tweets = {
                    'Tweet id': tweet.id,
                    'Contenido': tweet.rawContent,
                    'Fecha': tweet.date,
                    'Categoria': categoria,
                    'Autor username': tweet.user.username,
                    'Autor displayname': tweet.user.displayname,
                    'Autor location': tweet.user.location,
                    'Link': tweet.url,
                    'Likes': tweet.likeCount,
                    'Respuestas': tweet.replyCount,
                    'Retweets': tweet.retweetCount,
                    'Hashtags': tweet.hashtags,
                    'Mentioned users': tweet.mentionedUsers,
                    'Sentimiento':sentimiento,
                    'Emocion':emocion,
                    'Lang': tweet.lang
                }
            parsed_tweets.append(data_tweets)

            data_user = {
                'User id': tweet.user.id,
                'Autor username': tweet.user.username,
                'Autor displayname': tweet.user.displayname,
                'Autor location': tweet.user.location,
                'Fecha Creacion': tweet.user.created,
                'Link': tweet.user.url,
                'Verified':tweet.user.verified,
                'Followers': tweet.user.followersCount,
                'Following': tweet.user.friendsCount,
                'Tweets': tweet.user.statusesCount,
            }

            if tweet.user.id not in existing_ids:
                parsed_users.append(data_user)
                existing_ids.append(tweet.user.id)


    return parsed_tweets,parsed_users

In [28]:
async def scrape_and_save_sentiments(keywords,sentimiento,emocion,ruta_tweets='data/emociones/tweets.csv',ruta_users='data/emociones/users.csv',limit=1000):
    tweets= await get_top_tweets(keywords,limit)
    print("Se obtuvieron " + str(len(tweets)) + " Tweets de los keywords: " + keywords)
    
    parsed_tweets,parsed_user = parse_sentiment_tweets(tweets,sentimiento=sentimiento,emocion=emocion)
    print("Se parsearon correctamente los tweets")

    try_read_and_concat(ruta_tweets,parsed_tweets)
    try_read_and_concat(ruta_users,parsed_user)
    print("Se almaceno correctamente")

### Obtengo tweets por narrativa

In [ ]:
bronca_tweets = await get_top_tweets("Bronca",limit=2000)


In [39]:
len(bronca_tweets)

2148

In [36]:
for tweet in bronca_tweets:
    print(tweet.lang)

es
es
es
pt
pt
pt
es
es
en
es
es
es
es
es
pt
es
es
es
es
es
es
es
en
es
es
es
es
es
es
es
en
es
es
es
es
es
es
es
es
es
es
es
en
es
en
es
es
es
en
pt
es
es
es
en
es
es
es
es
es
es
es
es
es
es
ca
es
es
en
es
pt
es
es
es
es
es
en
es
es
es
es
es
es
es
es
es
es
es
es
es
pt
en
es
es
en
es
es
es
es
es
es
es
es
es
es
en
pt
pt
es
es
pt
es
es
es
en
es
es
en
es
es
es
en
es
es
es
pt
en
es
es
en
es
es
es
es
pt
es
es
pt
en
es
en
es
es
es
es
es
es
es
es
es
es
en
es
es
es
es
es
es
es
es
es
es
en
es
es
es
es
es
es
es
es
es
es
es
es
es
es
es
es
es
es
en
pt
pt
es
pt
es
es
es
es
en
es
es
es
es
pt
es
es
es
es
es
es
es
es
es
en
en
es
es
en
es
es
es
es
es
en
es
es
es
es
es
es
es
es
fr
es
es
es
es
es
en
es
es
en
es
es
es
es
es
en
es
es
pt
es
zxx
es
es
es
es
pt
es
es
es
es
es
es
en
es
en
zxx
es
es
es
es
en
en
es
es
es
es
es
es
es
es
es
es
es
pt
es
ar
es
es
en
es
es
pt
pt
en
es
es
en
es
es
es
es
en
es
es
es
pt
es
es
es
es
es
en
es
pt
es
es
es
es
es
en
es
es
es
es
es
es
es
es
es
es
es
es
es
en
es
es
es
es
ca
ca

In [40]:
parsed_tweets,parsed_users = parse_sentiment_tweets(bronca_tweets,"N","Enojo")

In [42]:
try_read_and_concat("data/emociones/tweets.csv",parsed_tweets)

In [43]:
try_read_and_concat("data/emociones/users.csv",parsed_users)


### Obtengo Emociones


#### Disgusto

In [44]:
await scrape_and_save_sentiments("Asco","N","Disgusto",limit=1500)


Se obtuvieron 346 Tweets de los keywords: Asco
Se parsearon correctamente los tweets
Se almaceno correctamente


In [45]:
await scrape_and_save_sentiments("desagradable","N","Disgusto",limit=1500)


Se obtuvieron 337 Tweets de los keywords: desagradable
Se parsearon correctamente los tweets
Se almaceno correctamente


In [46]:
await scrape_and_save_sentiments("nauseabundo","N","Disgusto",limit=1500)


Se obtuvieron 209 Tweets de los keywords: nauseabundo
Se parsearon correctamente los tweets
Se almaceno correctamente


In [47]:
await scrape_and_save_sentiments("vomitivo","N","Disgusto",limit=1500)


Se obtuvieron 159 Tweets de los keywords: vomitivo
Se parsearon correctamente los tweets
Se almaceno correctamente


In [53]:
await scrape_and_save_sentiments("espantoso","N","Disgusto",limit=1500)


Se obtuvieron 380 Tweets de los keywords: espantoso
Se parsearon correctamente los tweets
Se almaceno correctamente


In [54]:
await scrape_and_save_sentiments("asqueroso","N","Disgusto",limit=1500)


Se obtuvieron 275 Tweets de los keywords: asqueroso
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Enojo

In [60]:
await scrape_and_save_sentiments("forros","N","Enojo",limit=1500)


Se obtuvieron 347 Tweets de los keywords: forros
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Miedo

#### Alegria

In [62]:
await scrape_and_save_sentiments("felicidad","P","Alegria",limit=1500)


Se obtuvieron 449 Tweets de los keywords: felicidad
Se parsearon correctamente los tweets
Se almaceno correctamente


#### Tristeza

#### Sorpresa

#### Diversion

#### Gratitud

In [61]:
await scrape_and_save_sentiments("gracias","P","Gratitud",limit=1500)


Se obtuvieron 415 Tweets de los keywords: gracias
Se parsearon correctamente los tweets
Se almaceno correctamente


### Analizo emociones

In [63]:
df = pd.read_csv("data/emociones/tweets.csv")
df.shape

(5065, 16)

In [49]:
df.head()

,Tweet id,Contenido,Fecha,Categoria,Autor username,Autor displayname,Autor location,Link,Likes,Respuestas,Retweets,Hashtags,Mentioned users,Sentimiento,Emocion,Lang
0,1728609030967312739,que bronca no ser la persona favorita de nadie...,2023-11-26 02:58:03+00:00,Tweet,aagudz_,♰,en la ciudad del amor,https://twitter.com/aagudz_/status/17286090309...,37167,27,8475,[],[],N,Enojo,es
1,1728840105937559791,bueno pero sabes lo q mas bronca me da,2023-11-26 18:16:16+00:00,Tweet,nahiaraosudr,َ,NaN,https://twitter.com/nahiaraosudr/status/172884...,7308,16,1855,[],[],N,Enojo,es
2,1724880004213887244,#ElDeporteQueQueremos | Su apellido es sinónim...,2023-11-15 20:00:14+00:00,Tweet,Primicias,Primicias,Quito,https://twitter.com/Primicias/status/172488000...,1072,152,73,['ElDeporteQueQueremos'],[],N,Enojo,es
3,1727875035103142388,O qroier sendo o pai q da bronca no Pepito e o...,2023-11-24 02:21:25+00:00,Tweet,qguapoduer,bina 🦈,she⋆him,https://twitter.com/qguapoduer/status/17278750...,1233,4,163,[],[],N,Enojo,pt
4,1726708919853252890,que bronca me da https://t.co/B081WHz2tm,2023-11-20 21:07:41+00:00,Tweet,EzeChilo,EzeChilo,Buenos Aires,https://twitter.com/EzeChilo/status/1726708919...,1914,2,179,[],[],N,Enojo,pt


In [64]:
df['Lang'].value_counts()

Lang
es     4284
en      478
pt      234
und      18
zxx      10
ca        9
qme       6
it        6
fr        3
ja        2
in        2
art       2
qht       2
eu        1
ko        1
tl        1
qst       1
qam       1
de        1
ru        1
ar        1
lv        1
Name: count, dtype: int64

In [65]:
df_es = df[df['Lang'] == "es"]
df_es.shape

(4284, 16)

In [66]:
df_es['Emocion'].value_counts()

Emocion
Enojo       2076
Disgusto    1477
Alegria      386
Gratitud     345
Name: count, dtype: int64

In [67]:
df_es['Sentimiento'].value_counts()

Sentimiento
N    3553
P     731
Name: count, dtype: int64

In [7]:
autors = df['Autor username'].value_counts()
autors.head(n=20)

Autor username
eduardomenoni     4
fedetiberti       2
RDiosmel2         2
Karblack19        2
AnibalGarzon      2
carolina_moine    2
OldArg1810        2
evoespueblo       2
coralai_          2
ArgDiegote        2
CoralAINews       2
BizAnyio          2
alsersocimo       1
heroudr           1
ABCDigital        1
Jhoanromerofv     1
ARGprovincia25    1
lorstar08         1
MODO_Arg          1
ixXFelixXxi       1
Name: count, dtype: int64

### Analizo las narrativas